In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from helper_functions import *

In [ ]:
data = read_data()

In [ ]:
lonlat = getLonLatPairs(data, concat=True)

In [ ]:
lon_unique, lat_unique = getUniqueLonLat(lonlat)

In [ ]:
th_lon = smallestDiff(lon_unique, random=False)
th_lat = smallestDiff(lat_unique, random=False)
print((th_lon, th_lat, np.linalg.norm([th_lon, th_lat])))

We should be expecting (9.9999999747524271e-07, 9.9999999747524271e-07, 1.4142135588025491e-06) here. 

In [ ]:
rho = 3
th_nbr = rho * np.linalg.norm([th_lon, th_lat])

In [ ]:
from sklearn.cluster import MiniBatchKMeans as mbKMeans

In [ ]:
mbkm = mbKMeans(n_clusters=10, verbose=True)

In [ ]:
for k in sorted(data.keys()):
    mbkm.partial_fit(data[k].loc[:, ['r', 'g', 'b']].values)

In [ ]:
y = [0 for _ in range(10)]
x = range(10)

In [ ]:
mbkm.cluster_centers_

In [ ]:
plt.scatter(x, y, c=mbkm.cluster_centers_/255, s=1000)

In [ ]:
def getNeighbourList(arr, th):
    for row in arr:
        

In [ ]:
data[1].head()

In [ ]:
column_names = ['lon', 'lat', 'z', 'r', 'g', 'b', 'j', 'k', 'l']
data.columns = column_names
print('data.shape = {}'.format(data.shape))
data.head()

In [ ]:
data.describe().loc[['min', 'max'], ['lat','lon']]

In [ ]:
data_by_lon = data.groupby('lon')
lon_123_729523 = data_by_lon.get_group(-123.729523)
lon_123_729523_by_lat = lon_123_729523.groupby('lat')
lon_123_729523_lat_49_442196 = lon_123_729523_by_lat.get_group(49.442196)

In [ ]:
plt.scatter(lon_123_729523_lat_49_442196['j'].values, lon_123_729523_lat_49_442196['k'].values, c=lon_123_729523_lat_49_442196.loc[:,['r', 'g', 'b']].values/255, s=10000)